In [1]:
from faces.preprocessing import Videos2Datasets

Convert `videos` to `frames` and split them into train and test sets.

In [2]:
v2d = Videos2Datasets(
    videos=['../movie_faces_lukasz.MOV', '../movie_faces_anna.MOV'],
    people=['lukasz', 'anna'],
    train_dir='../datasets/train/',
    test_dir='../datasets/test/'
)
v2d.train_test_split(test_size=0.25)

Prepare `frames` with Transformer (or Transformers) provided by `faces`.

In [3]:
from faces.transformers import FaceLandmarksTransformer

Use model which will predict 68 face landmarks for given frame.

In [4]:
flt = FaceLandmarksTransformer('../models/shape_predictor_68_face_landmarks.dat')

In [5]:
X_train, y_train = flt.transform_from_directory(
    path_to_images='../datasets/train/',
    path_to_classes=['anna', 'lukasz'],
    landmarks_flatten_dimension=int(1/2 * 68 * (68 - 1))
)
X_valid, y_valid = flt.transform_from_directory(
    path_to_images='../datasets/test/',
    path_to_classes=['anna', 'lukasz'],
    landmarks_flatten_dimension=int(1/2 * 68 * (68 - 1))
)

1122it [03:22,  5.68it/s]
344it [01:06,  5.79it/s]


Train *simple* linear classifier (face recognizer will be used in the environment with highly constrained computational power).

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

/Users/lukaszambroziak/anaconda3/envs/mieszkai-base/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Measure performance of `logreg`.

In [7]:
roc_auc_score(y_train, logreg.predict_proba(X_train)[:, 1])

1.0

In [8]:
roc_auc_score(y_valid, logreg.predict_proba(X_valid)[:, 1])

1.0

Save model.

In [9]:
import joblib

joblib.dump(logreg, '../models/logreg_example_from_tutorial.joblib')

['../models/logreg_example_from_tutorial.joblib']